# TicTacToe Common

## Game Class

### TicTacToe

In [ ]:
import numpy as np
from enum import Enum
import re

class Role(Enum):
    NoRole = 0
    Player1 = 1
    Player2 = -1
    
class TicTacToe:
    def __init__(self, role1_enum, role2_enum, roleNone_enum):
        if role1_enum == role2_enum or role1_enum == roleNone_enum or role2_enum == roleNone_enum:
            raise Exception(f'[startGame] duplicated roles:{role1_enum},{role2_enum},{roleNone_enum}')
        
        self.size = 3
        self.lineLenToWin = self.size
        self.isInit = False
        self.totalSteps = self.size * self.size
        self.role1 = role1_enum
        self.role2 = role2_enum
        self.roleNone = roleNone_enum
        
    def startGame(self, role):
        if role != self.role1 and role != self.role2:
            raise Exception(f'[startGame] wrong role={role}')
        
        self.isInit = True
        self.roleTurn = role
        self.board = np.full(self.size * self.size, self.roleNone.value)
        self.winner = self.roleNone
        self.isEnded = False
        self.curStep = 0
        
    def makeStep(self, pos):
        if not self.isInit:
            raise Exception(f'[makeStep] game not started')
        if self.board[pos] != self.roleNone.value:
            raise Exception(f'[makeStep] position={pos} has already been taken by {self.board[pos]}') 
        if self.isEnded:
            print('[makeStep] game is ended')
            return

        self.curStep += 1
        self.board[pos] = self.roleTurn.value
        self.roleTurn = self.role2 if self.roleTurn == self.role1 else self.role1
        if self.checkWinner() or self.curStep == self.totalSteps:
            self.isEnded = True

    def checkSingleLineWinner(self, startPos, lineDotOffset):
        pos = startPos
        for i in range(1, self.lineLenToWin):
            pos += lineDotOffset
            if self.board[startPos] != self.board[pos]:
                return self.roleNone.value
        return self.board[startPos]

    def checkMultiLineWinner(self, startHeadPos, lineCount, lineHeadOffset, lineDotOffset):
        pos = startHeadPos
        for i in range(0, lineCount): 
            winner = self.checkSingleLineWinner(pos, lineDotOffset)
            pos += lineHeadOffset
            if winner != self.roleNone.value:
                self.winner = self.role1 if winner == self.role1.value else self.role2
                return True
        return False
            
    def checkWinner(self):
        return (
            # check rows line
            self.checkMultiLineWinner(
                startHeadPos=0, 
                lineCount=self.size, 
                lineHeadOffset=self.size, 
                lineDotOffset=1
            ) or
            # check columns line
            self.checkMultiLineWinner(
                startHeadPos=0, 
                lineCount=self.size, 
                lineHeadOffset=1, 
                lineDotOffset=self.size
            ) or
            # check '\' line
            self.checkMultiLineWinner(
                startHeadPos=0, 
                lineCount=1, 
                lineHeadOffset=0, 
                lineDotOffset=self.size + 1
            ) or
            # check '/' line
            self.checkMultiLineWinner(
                startHeadPos=self.size - 1, 
                lineCount=1, 
                lineHeadOffset=0, 
                lineDotOffset=self.size - 1
            )
        )

    def getEmptyPos(self):
        return np.where(self.board == 0)[0]

    def print(self):
        boardStr = ''
        for i in range(0, self.size):
            boardStr += '|'
            for j in range(0, self.size):
                role = self.board[i*self.size + j]
                symbol = ' '
                if role == self.role1.value:
                    symbol = 'O'
                elif role == self.role2.value:
                    symbol = 'X'
                boardStr += symbol + '|'
            boardStr += '\n'
        print('Game Board:')
        print(boardStr[:-1])
        print('Game Ended:', self.isEnded)
        print('Winner:', self.winner)

### Test Game

In [ ]:
b = TicTacToe(Role.Player1, Role.Player2, Role.NoRole)
b.startGame(Role.Player1)
b.makeStep(1)
b.makeStep(0)
b.makeStep(2)
b.makeStep(4)
b.makeStep(3)
b.makeStep(5)
b.makeStep(6)
b.makeStep(8)
b.makeStep(7)
b.print()

[makeStep] game is ended
Game Board:
|X|O|O|
|O|X|X|
|O| |X|
Game Ended: True
Winner: Role.Player2


## Common Agent Class

### Agent Base Class

In [ ]:
class AgentBase:
    def __init__(self, role):
        self.role = role

    def onEpisodeReset(self):
        pass
        
    def chosseAction(self, gameHandler):
        raise Exception('[chosseAction] not implemented')

    def updateValue(self, reward):
        pass

### Random Agent

In [ ]:
class RandAgent(AgentBase):
    def chosseAction(self, gameHandler):
        positions = gameHandler.getEmptyPos()
        idx = np.random.choice(len(positions))
        actionPos = positions[idx]
        return actionPos


### Human Agent

In [ ]:
class HumanAgent(AgentBase):
    def chosseAction(self, gameHandler):
        positions = gameHandler.getEmptyPos()
        while True:
            gameHandler.print()
            row = int(input("Input your action row:"))
            col = int(input("Input your action col:"))
            actionPos = row * gameHandler.size + col
            if actionPos in positions:
                return actionPos
            else:
                print(f'Position ({row},{col}) has already been taken! Try again.')

# Q-Learning Episode-Based (Monte Carlo)
更大范围的自举（Bootstrapping）：从根到叶子节点一整条线的自举

## Agent Class

In [ ]:
class AIAgent_QE(AgentBase):
    def __init__(self, role, lr=0.2, decayGamma=0.9, exploRate=0.3):
        super().__init__(role)
        self.learningRate = lr
        self.decayGamma = decayGamma
        self.exploRate = exploRate
        self.Q = {}
        self.decisions = []
        self.isTraining = True

    def getStateActionkey(self, boardState, actionPos):
        return str(boardState) + str(actionPos)
        
    def getStateActionValue(self, stateActionkey):
        value = self.Q.get(stateActionkey)
        if value is None:
            return 0;
        return value

    def setStateActionValue(self, stateActionkey, value):
        self.Q[stateActionkey] = value
        
    def chosseAction(self, gameHandler):
        positions = gameHandler.getEmptyPos()
        boardState = gameHandler.board
        
        if self.isTraining and np.random.uniform(0, 1) < self.exploRate:
            # take random action
            idx = np.random.choice(len(positions))
            actionPos = positions[idx]
        else:
            maxVal = -9999
            for pos in positions:
                stateActionkey = self.getStateActionkey(boardState, pos)
                value = self.getStateActionValue(stateActionkey)
                if value > maxVal:
                    maxVal = value
                    actionPos = pos

        stateActionkey = self.getStateActionkey(boardState, actionPos)
        self.decisions.append(stateActionkey)
        return actionPos

    def updateValue(self, reward):
        if not self.isTraining:
            return
        # only the final step has reward
        G_tNext = reward 
        reward = 0
        for stateActionkey in reversed(self.decisions):
            # G_t is the total accumulated reward from time step t
            G_t = reward + self.decayGamma * G_tNext # bellman equation
            Q_t = self.getStateActionValue(stateActionkey)
            Q_t += self.learningRate * (G_t - Q_t) # update Q with （weighted Monte Carlo method）
            self.setStateActionValue(stateActionkey, Q_t)
            G_tNext = Q_t # use Q_t or G_t(mean value or cur value)
        self.decisions = []    

## Eviroment Class

In [ ]:
class Enviroment:
    def __init__(self, game, agents):
        self.game = game
        self.agents = {}
        self.winRates = {}
        self.roles = []
        for agent in agents:
            self.winRates[agent.role] = 0
            self.agents[agent.role] = agent
            self.roles.append(agent.role)

    def getWinRateByRole(self, role):
        return self.winRates.get(role)
     
    def getAgentByRole(self, role):
        return self.agents.get(role)

    def playGame(self, episodes, onEpisodeEnd = None):
        winCount = {}
        for role in self.roles:
            self.winRates[role] = 0
            winCount[role] = 0

        episode = 0
        for episode in range(1, episodes+1):
            # restart the game
            startRole = self.roles[np.random.choice(len(self.roles))]# random role
            self.game.startGame(startRole);
            # play an episode
            while not self.game.isEnded:
                agent = self.agents[self.game.roleTurn]
                if agent is None:
                    raise Exception(f'[TrainingEnviroment] env has no agent with role={self.game.roleTurn}') 
                actionPos = agent.chosseAction(self.game)
                self.game.makeStep(actionPos)
            # update values
            for role in self.roles:
                # current agent is the winner
                if role is self.game.winner:
                    reward = 1
                    winCount[role] += 1
                    self.winRates[role] = winCount[role] / episode
                # draw
                elif self.game.winner is Role.NoRole:
                    reward = 0
                # current agent is the losser
                else:
                    reward = -1
                self.agents[role].updateValue(reward)
                
            if onEpisodeEnd != None:
                onEpisodeEnd(self, episode)

## Play the game

### Training with RandAgent

In [ ]:
def onEpisodeEnd(env, episode):
    if episode % 5000 != 0:
        return
    print(f'[episode = {episode}] AI1 win rate is {env.getWinRateByRole(Role.Player1)}')
    print(f'[episode = {episode}] RandAgent win rate is {env.getWinRateByRole(Role.Player2)}')

game = TicTacToe(Role.Player1, Role.Player2, Role.NoRole)
agent1 = AIAgent_QE(Role.Player1, lr=0.12, decayGamma=0.9, exploRate=0.4)
agent2 = RandAgent(Role.Player2)
trainingEnv = Enviroment(game, [agent1, agent2])
trainingEnv.playGame(60000, onEpisodeEnd)

[episode = 5000] AI1 win rate is 0.563
[episode = 5000] RandAgent win rate is 0.3163265306122449
[episode = 10000] AI1 win rate is 0.6098
[episode = 10000] RandAgent win rate is 0.26974276849164247
[episode = 15000] AI1 win rate is 0.6381517535671423
[episode = 15000] RandAgent win rate is 0.2424
[episode = 20000] AI1 win rate is 0.6559155915591559
[episode = 20000] RandAgent win rate is 0.228
[episode = 25000] AI1 win rate is 0.6694267770710829
[episode = 25000] RandAgent win rate is 0.21462575509061088
[episode = 30000] AI1 win rate is 0.6775
[episode = 30000] RandAgent win rate is 0.20646193858157447
[episode = 35000] AI1 win rate is 0.6856106063203612
[episode = 35000] RandAgent win rate is 0.19980570873453526
[episode = 40000] AI1 win rate is 0.692
[episode = 40000] RandAgent win rate is 0.19452986324658117
[episode = 45000] AI1 win rate is 0.6980377341718705
[episode = 45000] RandAgent win rate is 0.1895777777777778
[episode = 50000] AI1 win rate is 0.70348
[episode = 50000] Rand

### Training with another AI Agent
Best Parameters:
AIAgent_QE lr=0.12, decayGamma=0.9, exploRate=0.4 (best win rate against RandAgent is 0.9462)

In [ ]:
def onEpisodeEnd(env, episode):
    if episode % 5000 == 0:
        print(f'[episode = {episode}] AI1 win rate is {env.getWinRateByRole(Role.Player1)}')
        print(f'[episode = {episode}] AI2 win rate is {env.getWinRateByRole(Role.Player2)}')

game = TicTacToe(Role.Player1, Role.Player2, Role.NoRole)
agent1 = AIAgent_QE(Role.Player1, lr=0.1, decayGamma=0.9, exploRate=0.4)
agent2 = AIAgent_QE(Role.Player2, lr=0.1, decayGamma=0.9, exploRate=0.4)
trainingEnv = Enviroment(game, [agent1, agent2])
trainingEnv.playGame(60000, onEpisodeEnd)

[episode = 5000] AI1 win rate is 0.38015206082432973
[episode = 5000] AI2 win rate is 0.4097277822257806
[episode = 10000] AI1 win rate is 0.3703
[episode = 10000] AI2 win rate is 0.39965986394557823
[episode = 15000] AI1 win rate is 0.3760250683378892
[episode = 15000] AI2 win rate is 0.38826666666666665
[episode = 20000] AI1 win rate is 0.3697
[episode = 20000] AI2 win rate is 0.3866693334666733
[episode = 25000] AI1 win rate is 0.36817472698907955
[episode = 25000] AI2 win rate is 0.383085970316438
[episode = 30000] AI1 win rate is 0.36504867315642087
[episode = 30000] AI2 win rate is 0.3819
[episode = 35000] AI1 win rate is 0.3612960370296294
[episode = 35000] AI2 win rate is 0.3814653940675544
[episode = 40000] AI1 win rate is 0.3607590189754744
[episode = 40000] AI2 win rate is 0.3776
[episode = 45000] AI1 win rate is 0.36004444444444444
[episode = 45000] AI2 win rate is 0.37533613352001244
[episode = 50000] AI1 win rate is 0.35886
[episode = 50000] AI2 win rate is 0.372749819985

### Inference with RandAgent

when win rate is 0.9462, is hard for human to win

In [ ]:
def onEpisodeEnd3(env, episode):
    if episode % 5000 != 0:
        return
    print(f'[episode = {episode}] AI1 win rate is {env.getWinRateByRole(Role.Player1)}')
    print(f'[episode = {episode}] RandAgent win rate is {env.getWinRateByRole(Role.Player2)}')

agent1.isTraining = False
agent2 = RandAgent(Role.Player2)
infEnv = Enviroment(game, [agent1, agent2])
infEnv.playGame(5000, onEpisodeEnd3)

[episode = 5000] AI1 win rate is 0.9119823964792959
[episode = 5000] RandAgent win rate is 0.016908212560386472


### Inference with Human

In [ ]:
def onEpisodeEnd2(env, episode):
    env.game.print()
    print(f'[episode = {episode}] AI1 win rate is {env.getWinRateByRole(Role.Player1)}')
    print(f'[episode = {episode}] Your win rate is {env.getWinRateByRole(Role.Player2)}')

agent1.isTraining = False
agent2 = HumanAgent(Role.Player2)
infEnv = Enviroment(game, [agent1, agent2])
infEnv.playGame(5, onEpisodeEnd2)

In [ ]:
a = [1,5,3]
b = np.ones(a)
b

# Q-Learning Step-Based (Temporal Difference)
自举法Bootstrapping

## Agent Class

In [ ]:
class AIAgent_QS(AgentBase):
    def __init__(self, role, lr=0.2, decayGamma=0.9, exploRate=0.3):
        super().__init__(role)
        self.learningRate = lr
        self.decayGamma = decayGamma
        self.exploRate = exploRate
        self.Q = {}
        self.isTraining = True

    def onEpisodeReset(self):
        self.stateActionkey_t = None
        self.stateActionkey_tNext = None

    def getStateActionkey(self, boardState, actionPos):
        return str(boardState) + str(actionPos)
        
    def getStateActionValue(self, stateActionkey):
        value = self.Q.get(stateActionkey)
        if value is None:
            return 0;
        return value

    def setStateActionValue(self, stateActionkey, value):
        self.Q[stateActionkey] = value
        
    def chosseAction(self, gameHandler):
        positions = gameHandler.getEmptyPos()
        boardState = gameHandler.board

        actionPos = None
        if self.isTraining and np.random.uniform(0, 1) < self.exploRate:
            # take random action
            idx = np.random.choice(len(positions))
            actionPos = positions[idx]
        else:
            maxVal = -9999
            for pos in positions:
                stateActionkey = self.getStateActionkey(boardState, pos)
                value = self.getStateActionValue(stateActionkey)
                if value > maxVal:
                    maxVal = value
                    actionPos = pos

        self.stateActionkey_tNext = self.getStateActionkey(boardState, actionPos)
        return actionPos

    def updateValue(self, reward):
        if not self.isTraining:
            return

        if self.stateActionkey_t != None:
            Q_t = self.getStateActionValue(self.stateActionkey_t)
            Q_tNext = self.getStateActionValue(self.stateActionkey_tNext)
            
            G_t = reward + self.decayGamma * Q_tNext # bellman equation
            Q_t += self.learningRate * (G_t - Q_t) # update Q with Monte Carlo method
        
            self.setStateActionValue(self.stateActionkey_t, Q_t)
            
        self.stateActionkey_t = self.stateActionkey_tNext


## Eviroment Class

In [ ]:
class Enviroment:
    def __init__(self, game, agents):
        self.game = game
        self.agents = {}
        self.winRates = {}
        self.roles = []
        for agent in agents:
            self.winRates[agent.role] = 0
            self.agents[agent.role] = agent
            self.roles.append(agent.role)

    def getWinRateByRole(self, role):
        return self.winRates.get(role)
     
    def getAgentByRole(self, role):
        return self.agents.get(role)

    def playGame(self, episodes, onEpisodeEnd = None):
        winCount = {}
        for role in self.roles:
            self.winRates[role] = 0
            winCount[role] = 0

        episode = 0
        for episode in range(1, episodes+1):
            
            # restart the game
            startRole = self.roles[np.random.choice(len(self.roles))]# random role
            self.game.startGame(startRole);
            for role in self.roles:
                self.agents[role].onEpisodeReset()
                
            # play an episode
            while not self.game.isEnded:
                agent = self.agents[self.game.roleTurn]
                if agent is None:
                    raise Exception(f'[TrainingEnviroment] env has no agent with role={self.game.roleTurn}') 
                actionPos = agent.chosseAction(self.game)
                self.game.makeStep(actionPos)
                agent.updateValue(reward=0)

            # update value
            for role in self.roles:
                reward = 0
                # current agent is the winner
                if role == self.game.winner:
                    reward = 1
                    winCount[role] += 1
                    self.winRates[role] = winCount[role] / episode
                # current agent is the losser
                else:
                    reward = -1
                self.agents[role].updateValue(reward)
                
            if onEpisodeEnd != None:
                onEpisodeEnd(self, episode)

## Play the game

### Training with RandAgent

In [ ]:
def onEpisodeEnd(env, episode):
    if episode % 5000 != 0:
        return
    print(f'[episode = {episode}] AI1 win rate is {env.getWinRateByRole(Role.Player1)}')
    print(f'[episode = {episode}] RandAgent win rate is {env.getWinRateByRole(Role.Player2)}')

game = TicTacToe(Role.Player1, Role.Player2, Role.NoRole)
agent1 = AIAgent_QS(Role.Player1, lr=0.25, decayGamma=0.9, exploRate=0.5)
agent2 = RandAgent(Role.Player2)
trainingEnv = Enviroment(game, [agent1, agent2])
trainingEnv.playGame(120000, onEpisodeEnd)

[episode = 5000] AI1 win rate is 0.535
[episode = 5000] RandAgent win rate is 0.36208967173738993
[episode = 10000] AI1 win rate is 0.5656
[episode = 10000] RandAgent win rate is 0.334000200060018
[episode = 15000] AI1 win rate is 0.5875450060008001
[episode = 15000] RandAgent win rate is 0.3171333333333333
[episode = 20000] AI1 win rate is 0.5990799539976999
[episode = 20000] RandAgent win rate is 0.30835
[episode = 25000] AI1 win rate is 0.60872
[episode = 25000] RandAgent win rate is 0.30061202448097923
[episode = 30000] AI1 win rate is 0.6152333333333333
[episode = 30000] RandAgent win rate is 0.2962024472376888
[episode = 35000] AI1 win rate is 0.6206463041801195
[episode = 35000] RandAgent win rate is 0.2923428571428571
[episode = 40000] AI1 win rate is 0.6244
[episode = 40000] RandAgent win rate is 0.29053631703962995
[episode = 45000] AI1 win rate is 0.6289777777777777
[episode = 45000] RandAgent win rate is 0.28745749149829963
[episode = 50000] AI1 win rate is 0.63282531301252

### Training with another AI Agent
Best Parameters:
AIAgent_QS lr=0.12, decayGamma=0.9, exploRate=0.4 (best win rate against RandAgent is 0.9462)

In [ ]:
def onEpisodeEnd(env, episode):
    if episode % 5000 == 0:
        print(f'[episode = {episode}] AI1 win rate is {env.getWinRateByRole(Role.Player1)}')
        print(f'[episode = {episode}] AI2 win rate is {env.getWinRateByRole(Role.Player2)}')

game = TicTacToe(Role.Player1, Role.Player2, Role.NoRole)
agent1 = AIAgent_QS(Role.Player1, lr=0.25, decayGamma=0.9, exploRate=0.5)
agent2 = AIAgent_QS(Role.Player2, lr=0.25, decayGamma=0.9, exploRate=0.5)
trainingEnv = Enviroment(game, [agent1, agent2])
trainingEnv.playGame(120000, onEpisodeEnd)

[episode = 5000] AI1 win rate is 0.4446
[episode = 5000] AI2 win rate is 0.4288857771554311
[episode = 10000] AI1 win rate is 0.4479
[episode = 10000] AI2 win rate is 0.44096457874724837
[episode = 15000] AI1 win rate is 0.4482
[episode = 15000] AI2 win rate is 0.44642261785690474
[episode = 20000] AI1 win rate is 0.4467
[episode = 20000] AI2 win rate is 0.45101765264789717
[episode = 25000] AI1 win rate is 0.44309317118054165
[episode = 25000] AI2 win rate is 0.45632
[episode = 30000] AI1 win rate is 0.44553333333333334
[episode = 30000] AI2 win rate is 0.4562970864724315
[episode = 35000] AI1 win rate is 0.4447269921997771
[episode = 35000] AI2 win rate is 0.45977142857142855
[episode = 40000] AI1 win rate is 0.446275
[episode = 40000] AI2 win rate is 0.46124806240312016
[episode = 45000] AI1 win rate is 0.4467408938373669
[episode = 45000] AI2 win rate is 0.46328888888888886
[episode = 50000] AI1 win rate is 0.4485889717794356
[episode = 50000] AI2 win rate is 0.46418785127107626
[e

### Inference with RandAgent

when win rate is 0.9462, is hard for human to win

In [ ]:
def onEpisodeEnd3(env, episode):
    if episode % 5000 != 0:
        return
    print(f'[episode = {episode}] AI1 win rate is {env.getWinRateByRole(Role.Player1)}')
    print(f'[episode = {episode}] RandAgent win rate is {env.getWinRateByRole(Role.Player2)}')

agent1.isTraining = False
agent2 = RandAgent(Role.Player2)
infEnv = Enviroment(game, [agent1, agent2])
infEnv.playGame(5000, onEpisodeEnd3)

[episode = 5000] AI1 win rate is 0.8908
[episode = 5000] RandAgent win rate is 0.08494363929146538


### Inference with Human

In [ ]:
def onEpisodeEnd2(env, episode):
    env.game.print()
    print(f'[episode = {episode}] AI1 win rate is {env.getWinRateByRole(Role.Player1)}')
    print(f'[episode = {episode}] Your win rate is {env.getWinRateByRole(Role.Player2)}')

agent1.isTraining = False
agent2 = HumanAgent(Role.Player2)
infEnv = Enviroment(game, [agent1, agent2])
infEnv.playGame(5, onEpisodeEnd2)

Game Board:
| | | |
| | | |
| | | |
Game Ended: False
Winner: Role.NoRole


Input your action row: 1
Input your action col: 1


Game Board:
|O| | |
| |X| |
| | | |
Game Ended: False
Winner: Role.NoRole


Input your action row: 2
Input your action col: 2


Game Board:
|O| | |
|O|X| |
| | |X|
Game Ended: False
Winner: Role.NoRole


...

Game Board:
|X| | |
| |O| |
| | | |
Game Ended: False
Winner: Role.NoRole


Input your action row: 0
Input your action col: 1


Game Board:
|X|X|O|
| |O| |
| | | |
Game Ended: False
Winner: Role.NoRole


Input your action row: 2
Input your action col: 2


Game Board:
|X|X|O|
| |O| |
|O| |X|
Game Ended: True
Winner: Role.Player1
[episode = 2] AI1 win rate is 0.5
[episode = 2] Your win rate is 1.0
Game Board:
|O| | |
| | | |
| | | |
Game Ended: False
Winner: Role.NoRole
